In [ ]:
!pip install Keras==2.4.3
!pip install tensorflow==2.3.0
!pip install lime==0.1.1.32
!pip install nltk==3.2.4
!pip install autokeras
!pip install keras-tuner==1.0.2rc3

# LioNets: SMSSpam with TFIDF and Neural Networks -> Classification Task

In this notebook, we present how LioNets can be applied in predictive models using textual data with TFIDF vectorization.

In [ ]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
import keras
from keras.layers import Dense, LSTM, Dropout, Reshape
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import Input, Model
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.linear_model import Ridge, SGDRegressor, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score, balanced_accuracy_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from lime.lime_text import LimeTextExplainer
from lime.lime_tabular import LimeTabularExplainer #Maybe wont use
import pandas as pd
pd.set_option('max_colwidth',400)
import matplotlib.pyplot as plt
import re
import numpy as np
from collections import OrderedDict
import nltk
import seaborn as sns
from load_dataset import Load_Dataset
from LioNets import LioNet
from evaluation import Evaluation
import autokeras as ak
%matplotlib inline  

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

Load Spam SMS Dataset and split it 80-20 to train and test data

In [ ]:
X, y, class_names = Load_Dataset.load_smsspam()
X_train, X_valid, y_train, y_valid =  train_test_split(X,y,test_size=0.2, stratify = y, random_state=0)

Load few data to use them as unsupervised

In [ ]:
#X_unsup = Load_Dataset.load_unsupervised_data(True,True)

Setting the TFIDF vectorizer, with 1K vocabulary 

In [ ]:
vec = TfidfVectorizer(analyzer='word',max_features=1000)
vec.fit(X_train)
x_train = vec.transform(X_train).A
x_valid = vec.transform(X_valid).A
#x_unsup = vec.transform(X_unsup)
#x_train #input of predictor and encoder

Defining the input dimensions

In [ ]:
input_dim = len(vec.get_feature_names())
input_dim

Changing the format of the output from 0->0.1 and 1->0.9 in order to help the neural network with the sigmoid function

In [ ]:
train_y = [0.1 if i <=0.5 else 0.9 for i in y_train]
valid_y = [0.1 if i <=0.5 else 0.9 for i in y_valid]

We are setting the neural network architecture

In [ ]:
check_point = ModelCheckpoint("SMS_Predictor2020.hdf5", monitor="val_loss", verbose=2,save_best_only=True, mode="auto")
main_input = Input(shape=(input_dim,), dtype='float32', name='main_input')
x = Reshape((1,input_dim))(main_input)
x = LSTM(1000,activation='tanh')(x)
x = Dropout(0.75)(x)
x = Dense(500,activation='tanh')(x)
output_lay = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[main_input], outputs=[output_lay])
model.compile(optimizer="adam",loss=['binary_crossentropy'])

We train the model (we do not use unsupervised data yet)

In [ ]:
#model.fit(x_train, train_y, epochs=150, batch_size=64, shuffle=True, validation_data=(x_valid, valid_y), verbose=2, callbacks=[check_point])

We load the best model's weights

In [ ]:
weights_file = 'weights/SMS_Predictor.hdf5' # choose the best checkpoint few features
model.load_weights(weights_file) # load it
model.compile(loss="binary_crossentropy", optimizer=Adam())

In [ ]:
outputs_predictor = model.predict(x_train)

We are testing the performance of the predictor

In [ ]:
temp_pred = model.predict(x_train)
predictions = [0 if i[0] <=0.5 else 1 for i in temp_pred]
print('Train:',f1_score(y_train,predictions, average='macro'),f1_score(y_train,predictions, average='weighted'),
      balanced_accuracy_score(y_train,predictions),accuracy_score(y_train,predictions))

temp_pred = model.predict(x_valid)
predictions = [0 if i[0] <=0.5 else 1 for i in temp_pred]
print('Test:',f1_score(y_valid,predictions, average='macro'),f1_score(y_valid,predictions, average='weighted'),
      balanced_accuracy_score(y_valid,predictions), accuracy_score(y_valid,predictions))

We extract the encoder from the predictor

In [ ]:
encoder = Model(inputs=model.input, outputs=[model.layers[-2].output])
encoder.trainable = False
encoder.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=["accuracy"])

In [ ]:
output_layer = model.layers[-1]

In [ ]:
output_layer_weights = output_layer.get_weights()[0]

We encode our data, in order to build the decoder

In [ ]:
encoded_x_train = encoder.predict(x_train) #outputs of encoder / inputs of decoder
encoded_x_valid = encoder.predict(x_valid)
#encoded_x_unsup = encoder.predict(x_unsup)

### Decoder - Autokeras

In [ ]:
scaler = MinMaxScaler()
scaler.fit(encoded_x_train)
encoded_x_train_scaled = scaler.transform(encoded_x_train)

autodecoder = ak.StructuredDataRegressor(max_trials=10, loss='binary_crossentropy')
autodecoder.fit(encoded_x_train_scaled, x_train)

autodecoder = autodecoder.export_model()
autodecoder.summary()

In [ ]:
outputs_autodecoder = autodecoder.predict(encoded_x_train_scaled)
outputs_autodecoder

In [ ]:
autodecoder.evaluate(encoded_x_train_scaled, x_train)

In [ ]:
instances_evaluation = autodecoder.predict(encoded_x_train_scaled[:20]) 

In [ ]:
instances_evaluation_threshold = []
for r_m in instances_evaluation:
    a_t = [o if o > 0.045 else 0 for o in r_m]
    instances_evaluation_threshold.append(a_t)
inversed_decoded = vec.inverse_transform(instances_evaluation_threshold)
inversed_original = vec.inverse_transform(x_train[:20])
for i in range(len(inversed_original)):
    print('Original:',' '.join(sorted(inversed_original[i])))
    print(' Decoded:',' '.join(sorted(inversed_decoded[i])))
    print('""""""""""""""""""')

In [ ]:
instances_evaluation = autodecoder.predict(encoded_x_valid[:20])

In [ ]:
instances_evaluation_threshold = []
for r_m in instances_evaluation:
    a_t = [o if o > 0.045 else 0 for o in r_m]
    instances_evaluation_threshold.append(a_t)
inversed_decoded = vec.inverse_transform(instances_evaluation_threshold)
inversed_original = vec.inverse_transform(x_valid[:20])
for i in range(len(inversed_original)):
    print('Original:',' '.join(sorted(inversed_original[i])))
    print(' Decoded:',' '.join(sorted(inversed_decoded[i])))
    print('""""""""""""""""""')

We are building now the decoder

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
encoded_input = Input(shape=(encoded_x_train[0].shape))

x = Reshape((1,len(encoded_x_train[0])))(encoded_input)
x = LSTM(600, activation='tanh')(x)
x = Dropout(0.7)(x)
x = Dense(800, activation='tanh')(x)
decoded = Dense(input_dim, activation='sigmoid')(x)

decoder = Model(encoded_input,decoded)
decoder.compile(optimizer="Adam",loss=['binary_crossentropy'],metrics=[rmse,'mae'])

checkpoint_name = 'SMS_Decoder.hdf5' #or:'SMS_TFIDF_Decoder.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 2, save_best_only = True, mode ='auto')

We train the decoder, using some unsupervised data as well

In [ ]:
#decoder.fit(np.concatenate((encoded_x_train,encoded_x_unsup[12000:13000])), np.concatenate((x_train.toarray(),x_unsup[12000:13000].toarray())), epochs=250, batch_size=250, shuffle=True, validation_data=(encoded_x_valid,x_valid), verbose=2, callbacks=[checkpoint])

We load the best decoder

In [ ]:
weights_file = 'weights/SMS_Decoder.hdf5' # choose the best checkpoint few features
decoder.load_weights(weights_file) # load it
decoder.compile(optimizer="Adam",loss=['binary_crossentropy'],metrics=[rmse,'mae'])

In [ ]:
outputs_decoder = decoder.predict(encoded_x_train)
outputs_decoder

In [ ]:
decoder.evaluate(encoded_x_train,x_train)

In [ ]:
decoder.evaluate(encoded_x_valid,x_valid)

We evaluate some decoded instances manually (The first 10 sentences from the train and test set)

Train:

In [ ]:
instances_evaluation = decoder.predict(encoded_x_train[:5]) 

We will use a threshold to remove close to zero features

In [ ]:
instances_evaluation_threshold = []
for r_m in instances_evaluation:
    a_t = [o if o > 0.045 else 0 for o in r_m]
    instances_evaluation_threshold.append(a_t)
inversed_decoded = vec.inverse_transform(instances_evaluation_threshold)
inversed_original = vec.inverse_transform(x_train[:5])
for i in range(len(inversed_original)):
    print('Original:',' '.join(sorted(inversed_original[i])))
    print(' Decoded:',' '.join(sorted(inversed_decoded[i])))
    print('""""""""""""""""""')

Test:

In [ ]:
instances_evaluation = decoder.predict(encoded_x_valid[:5]) 

We will use a threshold to remove close to zero features

In [ ]:
instances_evaluation_threshold = []
for r_m in instances_evaluation:
    a_t = [o if o > 0.045 else 0 for o in r_m]
    instances_evaluation_threshold.append(a_t)
inversed_decoded = vec.inverse_transform(instances_evaluation_threshold)
inversed_original = vec.inverse_transform(x_valid[:5])
for i in range(len(inversed_original)):
    print('Original:',' '.join(sorted(inversed_original[i])))
    print(' Decoded:',' '.join(sorted(inversed_decoded[i])))
    print('""""""""""""""""""')

## LioNets Experiments
Having everything setted up, we are now ready to try our methodology. We first initialize LioNets. LioNets requires a predictor (the classifier itself), an encoder (extracted from the predictor), a decoder, as well as some data (for best results the training data, in order to push the neighbourhood generation through known distribution for the network)

In [ ]:
lionet = LioNet(model, decoder, encoder, x_train, decoder_lower_threshold=0.045, double_detector=True)

#lionet with autodecoder
autolionet = LioNet(model, autodecoder, encoder, x_train, decoder_lower_threshold=0.045, double_detector=True)

temp_instance.A[0] for a TFIDF array to be an array

## LioNets Experiments
Having everything setted up, we are now ready to try our methodology, Gradient x Input and LIME.

In [ ]:
import random 
#random.seed(2000)
random.seed(7777)
train = np.array(random.sample(X_train,200))#200
valid = np.array(random.sample(X_valid,200))
train.shape, valid.shape

Let's calculate the fidelity of Lime and LioNets

In [ ]:
split_expression = lambda s: re.split(r'\W+', s)
explainer = LimeTextExplainer(class_names=class_names, split_expression=split_expression)
def lime_predict(text):
    texts = vec.transform(text)
    a = model.predict(texts)
    b = 1 - a 
    return np.column_stack((b,a))

In [ ]:
def lime(text):
    explanation = explainer.explain_instance(text_instance=text, classifier_fn=lime_predict)
    weights = OrderedDict(explanation.as_list())
    lime_w = dict(zip(list(weights.keys()), list(weights.values())))
    weights = []
    keys = lime_w.keys()
    for f in vec.get_feature_names():
        if f in keys:
            weights.append(lime_w[f])
        else:
            weights.append(0)
    return np.array([weights]) #This is because lime interprets class with label 1

In [ ]:
def fi_lime(text):
    explanation = explainer.explain_instance(text_instance=text, classifier_fn=lime_predict)
    local_pred = explanation.local_pred[0]
    return local_pred #This is because lime interprets class with label 1
def fi_lionets(text):
    t_text = vec.transform(np.array([text]))[0].A[0]
    weights, res, loc_res = lionet.explain_instance(t_text,2000)
    return loc_res

In [ ]:
evaluator = Evaluation(model.predict,None,vec.transform,True)

In [ ]:
fidelity = evaluator.fidelity(train, [fi_lime, fi_lionets], class_n=0)
print('Lime fidelity:', fidelity[0][0])
print('LioNets fidelity:', fidelity[1][0])
fidelity = evaluator.fidelity(valid, [fi_lime, fi_lionets], class_n=0)
print('Lime fidelity:', fidelity[0][0])
print('LioNets fidelity:', fidelity[1][0])

Let's calculate non zero weights

In [ ]:
Xs = iutils.to_list(model.outputs)
softmax_found = False
ret = []
for x in Xs:
    layer, node_index, tensor_index = x._keras_history
    if checks.contains_activation(layer, activation="sigmoid"):
        softmax_found = True
        if isinstance(layer, keras.layers.Activation):
            ret.append(layer.get_input_at(node_index))
        else:
            layer_wo_act = innvestigate.utils.keras.graph.copy_layer_wo_activation(layer)
            ret.append(layer_wo_act(layer.get_input_at(node_index)))

In [ ]:
embedding_encoder = Sequential()
for i in range(0,2):
    print(model.layers[i])
    embedding_encoder.add(model.layers[i])
embedding_encoder.trainable = False
embedding_encoder.summary()

In [ ]:
model2 = Model(inputs=model.input, outputs=ret)
model2.trainable = False
model2.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=["accuracy"])

In [ ]:
analyzer = innvestigate.create_analyzer('input_t_gradient',model2)
analyzerLRP = innvestigate.create_analyzer('lrp.epsilon',model2)
def LRP_analyzer(X_t):
    ooo = analyzerLRP.analyze(X_t)[0]
    ooo = ooo*X_t #only on lrp
    return [ooo]

In [ ]:
feature_names = vec.get_feature_names()
def fi_GxI(text):
    t_text = vec.transform(np.array([text]))[0].A[0]
    ooo = analyzer.analyze(np.array([t_text]))[0]
    return [ooo][0]
def fi_LRP(text):
    t_text = vec.transform(np.array([text]))[0].A[0]
    ooo = analyzerLRP.analyze(np.array([t_text]))[0]
    ooo = ooo*t_text #only on lrp
    return [ooo][0]
def fi_lime(text):
    return lime(text)[0] #This is because lime interprets class with label 1
lionet = LioNet(model, decoder, encoder, x_train, decoder_lower_threshold=0.045, double_detector=True)
def fi_lionets(text):
    t_text = vec.transform(np.array([text]))[0].A[0]
    weights, res, loc_res = lionet.explain_instance(t_text,2000)
    nonzero = np.array([o if o > 0 else 0 for o in t_text])
    weights=weights*nonzero
    return weights

In [ ]:
non_zero = evaluator.non_zero_weights(train, [fi_GxI, fi_LRP, fi_lime, fi_lionets])
print('GxI Non Zero:', non_zero[0][0])
print('LRP Non Zero:', non_zero[1][0])
print('Lime Non Zero:', non_zero[2][0])
print('LioNets Non Zero:', non_zero[3][0])
non_zero = evaluator.non_zero_weights(valid, [fi_GxI, fi_LRP, fi_lime, fi_lionets])
print('GxI Non Zero:', non_zero[0][0])
print('LRP Non Zero:', non_zero[1][0])
print('Lime Non Zero:', non_zero[2][0])
print('LioNets Non Zero:', non_zero[3][0])

Let's calculate robustness

In [ ]:
robustness = evaluator.robustness(train,[fi_lime, fi_GxI, fi_LRP, fi_lionets], vec.get_feature_names())
print('GxI Robustness:', robustness[0])
print('LRP Robustness:', robustness[1])
print('Lime Robustness:', robustness[2])
print('LioNets Robustness:', robustness[3])
robustness = evaluator.robustness(valid,[fi_lime, fi_GxI, fi_LRP, fi_lionets], vec.get_feature_names())
print('GxI Robustness:', robustness[0])
print('LRP Robustness:', robustness[1])
print('Lime Robustness:', robustness[2])
print('LioNets Robustness:', robustness[3])

Let's calculate Altruist Score

In [ ]:
def fi_GxI(X_t, prediction, model):
    ooo = analyzer.analyze(np.array([X_t]))[0]
    return [ooo][0]
def fi_LRP(X_t, prediction, model):
    ooo = analyzerLRP.analyze(np.array([X_t]))[0]
    ooo = ooo*X_t #only on lrp
    return [ooo][0]
global ccount
ccount = 0
def fi_lime(t, prediction, model):
    global ccount
    expl = lime(train[ccount])[0]
    ccount = ccount+1
    return expl
def fi_lionets(X_t, prediction, model):
    weights, res, loc_red = lionet.explain_instance(X_t,2000)
    nonzero = np.array([o if o > 0 else 0 for o in X_t])
    #weights=weights*X_t
    #weights=weights*nonzero
    weights=weights
    return weights

In [ ]:
from altruist import Altruist
print("*Please let it run, it will take time probably*")
fi_names = {fi_GxI:'GxI',fi_LRP:'LRP',fi_lime:'Lime',fi_lionets:'LioNets'}
fis = [fi_GxI, fi_LRP, fi_lime,fi_lionets]
fis_scores = []
for i in fis:
    fis_scores.append([])
count = 0
feature_names 
X_t = vec.transform(train)
for instance in X_t:
    if (count + 1) % 100 == 0:
        print(count+1,"/",len(train),"..",end=", ")
    #print(len(instance))
    count = count + 1
    altruistino = Altruist(model, X_t.toarray(), fis, feature_names, None, True)
    untruthful_features = altruistino.find_untruthful_features(instance.toarray()[0])
    for i in range(len(untruthful_features[0])):
        fis_scores[i].append(len(untruthful_features[0][i]))
count = 0
print()
for fis_score in fis_scores:
    fi = fis[count]
    count = count + 1
    print(fi_names[fi],np.array(fis_score).mean())
fi_matrix = np.array(fis_scores)
count = 0
fi_all = []
for instance in X_t:
    fi_all.append(fi_matrix[:,count].min())
    count = count + 1
print("Altogether:",np.array(fi_all).mean())

In [ ]:
def fi_GxI(X_t, prediction, model):
    ooo = analyzer.analyze(np.array([X_t]))[0]
    return [ooo][0]
def fi_LRP(X_t, prediction, model):
    ooo = analyzerLRP.analyze(np.array([X_t]))[0]
    ooo = ooo*X_t #only on lrp
    return [ooo][0]
global ccount
ccount = 0
def fi_lime(t, prediction, model):
    global ccount
    expl = lime(valid[ccount])[0]
    ccount = ccount+1
    return expl
def fi_lionets(X_t, prediction, model):
    weights, res, loc_red = lionet.explain_instance(X_t,2000)
    nonzero = np.array([o if o > 0 else 0 for o in X_t])
    #weights=weights*X_t
    #weights=weights*nonzero
    weights=weights
    return weights

In [ ]:
from altruist import Altruist
print("*Please let it run, it will take time probably*")
fi_names = {fi_GxI:'GxI',fi_LRP:'LRP',fi_lime:'Lime',fi_lionets:'LioNets'}
fis = [fi_GxI, fi_LRP, fi_lime,fi_lionets]
fis_scores = []
for i in fis:
    fis_scores.append([])
count = 0
feature_names 
X_t = vec.transform(valid)
for instance in X_t:
    if (count + 1) % 100 == 0:
        print(count+1,"/",len(valid),"..",end=", ")
    #print(len(instance))
    count = count + 1
    altruistino = Altruist(model, X_t.toarray(), fis, feature_names, None, True)
    untruthful_features = altruistino.find_untruthful_features(instance.toarray()[0])
    for i in range(len(untruthful_features[0])):
        fis_scores[i].append(len(untruthful_features[0][i]))
count = 0
print()
for fis_score in fis_scores:
    fi = fis[count]
    count = count + 1
    print(fi_names[fi],np.array(fis_score).mean())
fi_matrix = np.array(fis_scores)
count = 0
fi_all = []
for instance in X_t:
    fi_all.append(fi_matrix[:,count].min())
    count = count + 1
print("Altogether:",np.array(fi_all).mean())

Let's calculate TI1

In [ ]:
evaluatorGxI = Evaluation(model.predict,analyzer.analyze,vec.transform,True)
evaluatorLRP = Evaluation(model.predict,LRP_analyzer,vec.transform,True)

Gradient x Input Results:

In [ ]:
e_results = evaluatorGxI.truthful_influence(train,fidelity=False)
print("Train TI1:",e_results[1]," NonZero:",e_results[0])
e_results = evaluatorGxI.truthful_influence(valid,fidelity=False)
print("Valid TI1:",e_results[1]," NonZero:",e_results[0])

LRP Results:

In [ ]:
e_results = evaluatorLRP.truthful_influence(train,fidelity=False)
print("Train TI1:",e_results[1]," NonZero:",e_results[0])
e_results = evaluatorLRP.truthful_influence(valid,fidelity=False)
print("Valid TI1:",e_results[1]," NonZero:",e_results[0])

Lime Results:

In [ ]:
evaluatorLIME = Evaluation(model.predict,lime,vec.transform,False,True)

In [ ]:
e_results = evaluatorLIME.truthful_influence(train,fidelity=False)
print("Train TI1:",e_results[1]," NonZero:",e_results[0])
e_results = evaluatorLIME.truthful_influence(valid,fidelity=False)
print("Valid TI1:",e_results[1]," NonZero:",e_results[0])

LioNets Results:

In [ ]:
def lionets_eval(text):
    weights, res, loc_red = lionet.explain_instance(text[0],2000)
    nonzero = np.array([o if o > 0 else 0 for o in text[0]])
    weights=weights*nonzero
    return [weights] #This is because lime interprets class with label 1

In [ ]:
evaluatorLioNets = Evaluation(model.predict,lionets_eval,vec.transform,True)

In [ ]:
e_results = evaluatorLioNets.truthful_influence(train,fidelity=False)
print("Train TI1:",e_results[1]," NonZero:",e_results[0])
e_results = evaluatorLioNets.truthful_influence(valid,fidelity=False)
print("Valid TI1:",e_results[1]," NonZero:",e_results[0])

In [ ]:
print("*Please let it run, it will take time probably*")
#for nne in [200,500,700,1000,1500,2000]:#,2000,2200,3000,4000,5000]:
for nne in [2000,2200,3000,4000,5000]:#,2000,2200,3000,4000,5000]:
    for tdc in [0,0.00001,0.01,0.02, 0.022, 0.035, 0.04, 0.045, 0.05, 0.07, 0.1, 0.15, 0.2, 0.22, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
        print(nne,tdc)
        feature_names = vec.get_feature_names()
        lionet = LioNet(model, decoder, encoder, x_train, decoder_lower_threshold=tdc, double_detector=True)
        def fi_lionets(X_t, prediction, model):
            weights, res, loc_red = lionet.explain_instance(X_t,nne)
            nonzero = np.array([o if o > 0 else 0 for o in X_t])
            #weights=weights*X_t
            #weights=weights*nonzero
            weights=weights
            return weights
        from altruist import Altruist
        fi_names = {fi_GxI:'GxI',fi_LRP:'LRP',fi_lime:'Lime',fi_lionets:'LioNets'}
        fis = [fi_lionets]#fi_lionets
        fis_scores = []
        for i in fis:
            fis_scores.append([])
        count = 0
        feature_names 
        X_t = vec.transform(train)
        for instance in X_t:
            if (count + 1) % 100 == 0:
                print(count+1,"/",len(train),"..",end=", ")
            #print(len(instance))
            count = count + 1
            altruistino = Altruist(model, X_t.toarray(), fis, feature_names, None, True)
            untruthful_features = altruistino.find_untruthful_features(instance.toarray()[0])
            for i in range(len(untruthful_features[0])):
                fis_scores[i].append(len(untruthful_features[0][i]))
        print()
        count = 0
        for fis_score in fis_scores:
            fi = fis[count]
            count = count + 1
            print(fi_names[fi],np.array(fis_score).mean())

## Qualitative:
Let's see an example manually:

In [ ]:
temp_instance = x_train[9].copy()
transparent_model = Ridge(alpha=0.0001,fit_intercept=True,random_state=0)
weights, real_prediction, local_prediction = lionet.explain_instance(temp_instance[0:],2000, transparent_model)

In [ ]:
str('Sentence: "' + X_train[9] + '"   Class: ' + str(train_y[9]))

In [ ]:
weights, real_prediction, local_prediction = lionet.explain_instance(x_train[9][0:], 2000, transparent_model)
print("Real prediction:",real_prediction,", Local prediction:",local_prediction)

In [ ]:
model_weights = pd.DataFrame({"Features": list(vec.get_feature_names()), 
                              "Features' Weights": list(weights*x_train[9][0:])})
model_weights = model_weights.sort_values(by="Features' Weights", ascending=False)
model_weights = model_weights[(model_weights["Features' Weights"] != 0)]    
#model_weights, lime_predict([text])[0][1], rd.predict(texts)[0], weights
plt.figure(num=None, figsize=(4, 3), dpi=200, facecolor='w', edgecolor='k')
sns.barplot(x="Features' Weights", y="Features", data=model_weights)
#plt.xticks(rotation=90)
#plt.title(str('Features not appearing in the instance'))
plt.show()

In [ ]:
np.argmax(weights),np.argmin(weights),np.max(weights),np.min(weights),vec.get_feature_names()[np.argmax(weights)],vec.get_feature_names()[np.argmin(weights)]

In [ ]:
lime_predict(['this congrat treat pend am not on mail for day wil mail onc thru respect mother at home check mail'])[0][1]

In [ ]:
counter_weights = []
counter_features = []
for i in range(len(weights)):
    if weights[i]!=0:
        if vec.get_feature_names()[i] not in X_train[9]:
            counter_weights.append(weights[i])
            counter_features.append(vec.get_feature_names()[i])
co_weights = pd.DataFrame({"Counter Features": list(counter_features), 
                                  "Features' Weights": list(counter_weights)})
co_weights = co_weights.sort_values(by="Features' Weights", ascending=False)
co_weights = pd.concat([co_weights.head(5),co_weights.tail(5)])
plt.figure(num=None, figsize=(4, 3), dpi=200, facecolor='w', edgecolor='k')
sns.barplot(x="Features' Weights", y="Counter Features", data=co_weights)
#plt.xticks(rotation=90)
#plt.title(str('Features not appearing in the instance'))
plt.show()

In [ ]:
for i in range(len(X_train)):
    if 'teach' in X_train[i]:
        print(i)

In [ ]:
X_train[119]

In [ ]:
weights, real_prediction, local_prediction = lionet.explain_instance(x_train[119][0:], 2000, transparent_model)

In [ ]:
model_weights = pd.DataFrame({"Features": list(vec.get_feature_names()), 
                                "Features' Weights": list(weights*x_train[119][0:])})
model_weights = model_weights.sort_values(by="Features' Weights", ascending=False)
model_weights = model_weights[(model_weights["Features' Weights"] != 0)]    
#model_weights, lime_predict([text])[0][1], rd.predict(texts)[0], weights
plt.figure(num=None, figsize=(4, 3), dpi=200, facecolor='w', edgecolor='k')
sns.barplot(x="Features' Weights", y="Features", data=model_weights)
#plt.xticks(rotation=90)
#plt.title(str('Features not appearing in the instance'))
plt.show()

### Qualitative using autodecoder:

In [ ]:
temp_instance = x_train[9].copy()
transparent_model = Ridge(alpha=0.0001,fit_intercept=True,random_state=0)
weights, real_prediction, local_prediction = autolionet.explain_instance(temp_instance[0:],2000, transparent_model)

In [ ]:
str('Sentence: "' + X_train[9] + '"   Class: ' + str(train_y[9]))

In [ ]:
weights, real_prediction, local_prediction = autolionet.explain_instance(x_train[9][0:], 2000, transparent_model)
print("Real prediction:",real_prediction,", Local prediction:",local_prediction)

In [ ]:
model_weights = pd.DataFrame({"Features": list(vec.get_feature_names()), 
                              "Features' Weights": list(weights*x_train[9][0:])})
model_weights = model_weights.sort_values(by="Features' Weights", ascending=False)
model_weights = model_weights[(model_weights["Features' Weights"] != 0)]    
#model_weights, lime_predict([text])[0][1], rd.predict(texts)[0], weights
plt.figure(num=None, figsize=(4, 3), dpi=200, facecolor='w', edgecolor='k')
sns.barplot(x="Features' Weights", y="Features", data=model_weights)
#plt.xticks(rotation=90)
#plt.title(str('Features not appearing in the instance'))
plt.show()

In [ ]:
np.argmax(weights),np.argmin(weights),np.max(weights),np.min(weights),vec.get_feature_names()[np.argmax(weights)],vec.get_feature_names()[np.argmin(weights)]

In [ ]:
counter_weights = []
counter_features = []
for i in range(len(weights)):
    if weights[i]!=0:
        if vec.get_feature_names()[i] not in X_train[9]:
            counter_weights.append(weights[i])
            counter_features.append(vec.get_feature_names()[i])
co_weights = pd.DataFrame({"Counter Features": list(counter_features), 
                                  "Features' Weights": list(counter_weights)})
co_weights = co_weights.sort_values(by="Features' Weights", ascending=False)
co_weights = pd.concat([co_weights.head(5),co_weights.tail(5)])
plt.figure(num=None, figsize=(4, 3), dpi=200, facecolor='w', edgecolor='k')
sns.barplot(x="Features' Weights", y="Counter Features", data=co_weights)
#plt.xticks(rotation=90)
#plt.title(str('Features not appearing in the instance'))
plt.show()

In [ ]:
for i in range(len(X_train)):
    if 'teach' in X_train[i]:
        print(i)

In [ ]:
X_train[119]

In [ ]:
weights, real_prediction, local_prediction = autolionet.explain_instance(x_train[119][0:], 2000, transparent_model)

In [ ]:
model_weights = pd.DataFrame({"Features": list(vec.get_feature_names()), 
                              "Features' Weights": list(weights*x_train[119][0:])})
model_weights = model_weights.sort_values(by="Features' Weights", ascending=False)
model_weights = model_weights[(model_weights["Features' Weights"] != 0)]    
#model_weights, lime_predict([text])[0][1], rd.predict(texts)[0], weights
plt.figure(num=None, figsize=(4, 3), dpi=200, facecolor='w', edgecolor='k')
sns.barplot(x="Features' Weights", y="Features", data=model_weights)
#plt.xticks(rotation=90)
#plt.title(str('Features not appearing in the instance'))
plt.show()